# ARIMA Model

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import torch

src_path = os.path.abspath(os.path.join('../../..'))
if src_path not in sys.path:
    sys.path.append(src_path)

from src.models.arima import MultiARIMA
from src.utils.datasets import HSMD

In [ ]:
dataset = HSMD('../../../data/HSMD', stocks=['nflx'], columns=['Close'], window_size=1)

In [ ]:
generator = torch.Generator().manual_seed(1)
train_size = int(len(dataset) * 0.9)
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size], generator=generator)

In [ ]:
history = [x.astype(float).flatten() for x in train_set]

In [ ]:
model = ARIMA(history, order=(5, 1, 0))
fit = model.fit()
print(fit.summary())

In [ ]:
residuals = pd.DataFrame(fit.resid)
residuals.plot()
plt.show()
# density plot of residuals
residuals.plot(kind='kde')
plt.show()

In [ ]:
predictions = []
gt = []
for window in test_set:
    window = window.astype(float).flatten()
    model = ARIMA(history, order=(5, 1, 0))
    fit = model.fit()
    output = fit.forecast()
    predictions.append(output)
    gt.append(window)
    history.append(window)

In [ ]:
time = np.arange(0, len(gt))
plt.plot(time, gt, 'g')
plt.plot(time, predictions, 'b')
plt.show()